<a href="https://colab.research.google.com/github/alpadedhia/ai-agents/blob/main/Langgraph_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai python-dotenv langchain langgraph langchain_openai langchain_community langchain_core

In [22]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI


from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [20]:
client = OpenAI(api_key="")

In [25]:
os.environ["TAVILY_API_KEY"] = ""

tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [26]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [27]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

# prompt = """
# You run in a loop of Thought, Action, PAUSE, Observation.
# At the end of the loop you output an Answer
# Use Thought to describe your thoughts about the question you have been asked.
# Use Action to run one of the actions available to you - then return PAUSE.
# Observation will be the result of running those actions.

# Your available actions are:

# calculate:
# e.g. calculate: 4 * 7 / 3
# Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

# average_dog_weight:
# e.g. average_dog_weight: Collie
# returns average weight of a dog when given the breed

# Example session:

# Question: How much does a Bulldog weigh?
# Thought: I should look the dogs weight using average_dog_weight
# Action: average_dog_weight: Bulldog
# PAUSE

# You will be called again with this:

# Observation: A Bulldog weights 51 lbs

# You then output:

# Answer: A bulldog weights 51 lbs
# """.strip()


prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo", api_key="")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)


In [32]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

result['messages'][-1].content

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_NCeqeLHgqvJO8nH2u0kkiCxn', 'type': 'tool_call'}
Back to the model!


'The current weather in San Francisco is partly cloudy with a temperature of 64.0°F (17.8°C). The wind speed is 3.1 mph (5.0 kph) coming from the north-northeast direction. The humidity is at 78%, and there is no precipitation at the moment.'

In [34]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o", api_key="")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

result['messages'][-1].content

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl winner 2024'}, 'id': 'call_qpB3omgzD2TVLIm9LIKvAYjA', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_lY5pqruTxHabPCNGKfh0hFtk', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current GDP of Missouri 2024'}, 'id': 'call_B9GJUFfxFvHmLCotXuSMyYXn', 'type': 'tool_call'}
Back to the model!


"1. **Who won the Super Bowl in 2024?**\n   - The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers 25-22 in overtime.\n\n2. **In what state is the winning team headquarters located?**\n   - The Kansas City Chiefs' headquarters is located in Missouri.\n\n3. **What is the GDP of that state?**\n   - The real GDP for Missouri in the 3rd quarter of 2023 was $52.8 billion for the real estate sector, which was the highest in the state. For a more comprehensive figure, the state's GDP would encompass all sectors, but specific data for the entire GDP for 2024 isn't provided in the available sources directly. However, the U.S. Bureau of Economic Analysis provides updates, and Missouri's economy has been growing at an annual rate of around 3.0 percent as of the second quarter of 2024."

In [31]:
# class Agent:
#     def __init__(self, system=""):
#         self.system = system
#         self.messages = []
#         if self.system:
#             self.messages.append({"role": "system", "content": system})

#     def __call__(self, message):
#         self.messages.append({"role": "user", "content": message})
#         result = self.execute()
#         self.messages.append({"role": "assistant", "content": result})
#         return result

#     def execute(self):
#         completion = client.chat.completions.create(
#                         model="gpt-4o",
#                         temperature=0,
#                         messages=self.messages)
#         return completion.choices[0].message.content



# def calculate(what):
#     return eval(what)

# def average_dog_weight(name):
#     if name in "Scottish Terrier":
#         return("Scottish Terriers average 20 lbs")
#     elif name in "Border Collie":
#         return("a Border Collies average weight is 37 lbs")
#     elif name in "Toy Poodle":
#         return("a toy poodles average weight is 7 lbs")
#     else:
#         return("An average dog weights 50 lbs")

# known_actions = {
#     "calculate": calculate,
#     "average_dog_weight": average_dog_weight
# }

In [14]:
# action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

# def query(question, max_turns=5):
#     i = 0
#     bot = Agent(prompt)
#     next_prompt = question
#     while i < max_turns:
#         i += 1
#         result = bot(next_prompt)
#         print(result)
#         actions = [
#             action_re.match(a)
#             for a in result.split('\n')
#             if action_re.match(a)
#         ]
#         if actions:
#             # There is an action to run
#             action, action_input = actions[0].groups()
#             if action not in known_actions:
#                 raise Exception("Unknown action: {}: {}".format(action, action_input))
#             print(" -- running {} {}".format(action, action_input))
#             observation = known_actions[action](action_input)
#             print("Observation:", observation)
#             next_prompt = "Observation: {}".format(observation)
#         else:
#             return

In [ ]:
# question = """I have 2 dogs, a border collie and a scottish terrier. \
# What is their combined weight"""
# query(question)